# Get data

This notebook gets the data


In [1]:
import pandas as pd
import json
import yaml
import numpy as np
import requests
import re

## 1. EuropePMC

(Explain)

In [2]:
# Define the query parameters
USER_AGENT = "Mozilla/5.0"
page_size = 1000
cursor_mark = '*'
format_type = 'json'
base_url_text = "https://www.ebi.ac.uk/europepmc/webservices/rest/{}/fullTextXML"

In [3]:
import requests

# Initialize the list to store retracted articles
retracted_articles_ePMC = []

print("Initial request to Europe PMC to get the retracted paper JSON")
# Make the initial request to get the total number of results
url = f'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PUB_TYPE%3A%22retraction%20of%20publication%22&resultType=idlist&cursorMark={cursor_mark}&pageSize=1&format={format_type}'

try:
    response = requests.get(url).json()
    total = response['hitCount']

    # Calculate the number of requests needed to retrieve all results
    num_requests = (total + page_size - 1) // page_size
    print(f"Will need to perform {num_requests} requests.")

    # Iterate through each page and append the results to the list
    for i in range(num_requests):
        cursor_mark = response['nextCursorMark']

        if cursor_mark is None:
            break
        url = f'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PUB_TYPE%3A%22retraction%20of%20publication%22&resultType=idlist&cursorMark={cursor_mark}&pageSize={page_size}&format={format_type}'
        print(f"Request {i + 1} of {num_requests}: {url}")

        try:
            response = requests.get(url).json()
            retracted_articles_ePMC.extend(response['resultList']['result'])
        except Exception as e:
            #print("An error occurred: " + str(e))
            pass

except Exception as e:
    #print("An error occurred: " + str(e))
    pass


Initial request to Europe PMC to get the retracted paper JSON
Will need to perform 15 requests.
Request 1 of 15: https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PUB_TYPE%3A%22retraction%20of%20publication%22&resultType=idlist&cursorMark=AoIIQMGRICg0NzYzMTA2Mw==&pageSize=1000&format=json
Request 2 of 15: https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PUB_TYPE%3A%22retraction%20of%20publication%22&resultType=idlist&cursorMark=AoIIQGaFvCg0NzM2MjYwOQ==&pageSize=1000&format=json
Request 3 of 15: https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PUB_TYPE%3A%22retraction%20of%20publication%22&resultType=idlist&cursorMark=AoIIQESciig0NjI5NzkzOQ==&pageSize=1000&format=json
Request 4 of 15: https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PUB_TYPE%3A%22retraction%20of%20publication%22&resultType=idlist&cursorMark=AoIIQB76bCg0NDg0MDYwMQ==&pageSize=1000&format=json
Request 5 of 15: https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=P

Get full text when available

In [4]:
for i in range(len(retracted_articles_ePMC)):
    entry = retracted_articles_ePMC[i]
    if "fullTextIdList" in entry.keys():
        id = entry["fullTextIdList"]["fullTextId"][0]
        url = base_url_text.format(id)
        retracted_articles_ePMC[i]["fullTextURL"] = url
        # Request text
        response = requests.get(url, headers={"User-Agent": USER_AGENT})
        response_code = response.status_code

        try:
            if response_code < 200 or response_code >= 300:
                #print(f"Failed to retrieve data for {id}. Response code: " + str(response_code))
                # Sometimes an ID not available through the REST API is available as a normal link, e.g., https://europepmc.org/article/pmc/PMC10073366 ... Seems to be the case for several ids. Shame!
                pass
            else:
                response_text = response.text
                body_content = ""
                regex = "<body(.*?)</body"
                pattern = re.compile(regex, re.DOTALL)
                matcher = pattern.search(response_text)
                if matcher:
                    body_content = matcher.group(1)

                    cleaned_text = re.sub("<.*?>", "", body_content)  # remove  all XML tags
                    cleaned_text = re.sub("&[#\\w]+?;", "", cleaned_text)  # remove HTML entities
                    cleaned_text = re.sub("\\s+", " ", cleaned_text)  # replace multiple whitespaces with a single space
                    cleaned_text = cleaned_text.replace(">", "") # left from the body tag if <body>
                    cleaned_text = cleaned_text.strip()  #  remove leading/trailing whitespaces
                    retracted_articles_ePMC[i]["fullText"] = cleaned_text

        except Exception as e:
            #print(f"An error occurred for {id}: " + str(e))
            pass



In [5]:
# Create the final JSON object and save to file
output_json = json.dumps(retracted_articles_ePMC)
with open("../../../data/ePMC/retracts.json", "w") as f:
    f.write(output_json)

In [6]:
%load_ext watermark
%watermark -v -m 

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 5.19.0-41-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

